In [28]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

In [30]:
import clip
import torch
from PIL import Image


image = Image.open('assets/chair.png')

def test(caption, image=image):
    device = "cuda"
    
    model, transform = clip.load("ViT-B/32", device=device)
    image = transform(image).unsqueeze(0).to(device)
    text = clip.tokenize(['a ' +caption]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

    similarity_score = (image_features @ text_features.T).mean()
    return similarity_score


In [32]:
test('chair'), test('dog'), test('lego'), test('desktop'), test('person'), test('object')

(tensor(31.5312, device='cuda:0', dtype=torch.float16),
 tensor(25.2031, device='cuda:0', dtype=torch.float16),
 tensor(25.4844, device='cuda:0', dtype=torch.float16),
 tensor(22.9062, device='cuda:0', dtype=torch.float16),
 tensor(24.8906, device='cuda:0', dtype=torch.float16),
 tensor(29.6250, device='cuda:0', dtype=torch.float16))

In [31]:
test('funiture')

tensor(28.3750, device='cuda:0', dtype=torch.float16)

In [29]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
import subprocess

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to(device)
        

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
def test_blip(image):

    inputs = processor(image, return_tensors="pt").to(device, torch.float16)

    generated_ids = model.generate(**inputs, max_new_tokens=20)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    return generated_text
